# Retrieval Augmented Generation (RAG)<br>
<ul>
Indexing<ol>
Doccument loading<br>
Text Splitting<br>
Embedding<br>
vector storeage<br></ol>
Retirval<ol>
query embedding<br>
Sim search<br>
retirve K docc K-means<br>
Context augmentation<br></ol>
</ul>


### Data loaders

In [9]:
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
import os

loader= TextLoader("data/text.txt",encoding="utf-8")
documents = loader.load()

print(f"Loaded {len(documents)} documents from text file.")
print("metadata:", documents[0].metadata)

Loaded 1 documents from text file.
metadata: {'source': 'data/text.txt'}


In [10]:
loader = PyPDFLoader("data/pdf.pdf")
documents = loader.load()
print(f"Loaded {len(documents)} documents from PDF file.")
print("metadata:", documents[0].metadata)

Loaded 3 documents from PDF file.
metadata: {'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': 'Smart Water Bottle Proposal', 'author': 'ChatGPT Canvas', 'source': 'data/pdf.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}


In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/William_Hanna")
documents = loader.load()
print(f"Loaded {len(documents)} documents from web page.")
print("metadata:", documents[0].metadata)

Loaded 1 documents from web page.
metadata: {'source': 'https://en.wikipedia.org/wiki/William_Hanna', 'title': 'William Hanna - Wikipedia', 'language': 'en'}


### Text splitter

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

long_text = """
LangChain is a powerful framework for developing applications powered by language models.
It enables applications that are:
1. Data-aware: connect a language model to other sources of data.
2. Agentic: allow a language model to interact with its environment.

The core idea of LangChain is to "chain" together different components to create more advanced use cases around LLMs.
This includes modules for:
- Models: LLMs, ChatModels, Embeddings
- Prompts: PromptTemplate, ChatPromptTemplate
- Output Parsers: StrOutputParser, JsonOutputParser, PydanticOutputParser
- Indexes: Document Loaders, Text Splitters, Vectorstores, Retrievers
- Chains: Combining components with LCEL
- Agents: LLMs that can make decisions and use tools
- Memory: Persisting state between turns

LangChain is available in Python and JavaScript/TypeScript.
It also has related projects like LangServe for deployment and LangSmith for observability.
"""
doc_to_split = Document(page_content=long_text, metadata={"source": "internal_doc"})

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=20,separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents([doc_to_split])

print(f"Split document into {len(chunks)} chunks:")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk.page_content[:50]}... (metadata: {chunk.metadata})")

Split document into 13 chunks:
Chunk 1: LangChain is a powerful framework for developing a... (metadata: {'source': 'internal_doc'})
Chunk 2: It enables applications that are:
1. Data-aware: c... (metadata: {'source': 'internal_doc'})
Chunk 3: 2. Agentic: allow a language model to interact wit... (metadata: {'source': 'internal_doc'})
Chunk 4: The core idea of LangChain is to "chain" together ... (metadata: {'source': 'internal_doc'})
Chunk 5: more advanced use cases around LLMs.... (metadata: {'source': 'internal_doc'})
Chunk 6: This includes modules for:
- Models: LLMs, ChatMod... (metadata: {'source': 'internal_doc'})
Chunk 7: - Prompts: PromptTemplate, ChatPromptTemplate... (metadata: {'source': 'internal_doc'})
Chunk 8: - Output Parsers: StrOutputParser, JsonOutputParse... (metadata: {'source': 'internal_doc'})
Chunk 9: - Indexes: Document Loaders, Text Splitters, Vecto... (metadata: {'source': 'internal_doc'})
Chunk 10: - Chains: Combining components with LCEL
- Agents:... (metad